In [ ]:
pip install xgboost # xgboost 설치

## 산타데르 제품 추천

In [ ]:
# 라이브러리
import pandas as pd
import numpy as np
import xgboost as xgb

In [ ]:
# 시드값 고정
np.random.seed(2018)

In [ ]:
# 데이터 로드
trn = pd.read_csv('train_ver2.csv')
tst = pd.read_csv('test_ver2.csv')

KeyboardInterrupt: 

In [ ]:
trn.head(5)

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,2015-01-28,1375586,N,ES,H,35,2015-01-12,0.0,6,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
1,2015-01-28,1050611,N,ES,V,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
2,2015-01-28,1050612,N,ES,V,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
3,2015-01-28,1050613,N,ES,H,22,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
4,2015-01-28,1050614,N,ES,V,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0


### 1. 데이터 전처리

In [ ]:
# 베이스라인의 간소화를 위해 2016년만 사용
trn['fecha_dato_dt'] = pd.to_datetime(trn['fecha_dato'])
trn = trn[trn['fecha_dato_dt'].dt.year == 2016]
trn.drop(['fecha_dato_dt'], axis=1, inplace=True)

In [ ]:
# 제품 변수 따로 저장
prods = trn.columns[24:].tolist()

# 제품 변수 결측치 0으로 대체
trn[prods] = trn[prods].fillna(0.0).astype(np.int8)

In [ ]:
# 24개의 제품 변수 모두 0인 데이터 삭제
no_product = trn[prods].sum(axis=1)==0
trn = trn[~no_product]

In [ ]:
# 테스트 셋과 합치기, 테스트셋에 없는 제품변수는 0으로 대체
for col in trn.columns[24:]:
    tst[col] = 0
df = pd.concat([trn,tst],axis=0)

In [ ]:
# 학습에 사용할 변수 담는 list
features = []

# 범주형 변수의 label encoding (결측치에는 -99 대입)
categorical_cols = ['ind_empleado','pais_residencia','sexo','tiprel_1mes','indresi','indext','conyuemp','canal_entrada','indfall','tipodom','nomprov','segmento']
for col in categorical_cols:
    df[col], _ = df[col].factorize(na_sentinel=-99) # 정수로 인코딩된 값, 인코딩된 범주

features += categorical_cols

In [ ]:
# 수치형 변수의 결측치 처리 및 타입 변환
df['age'].replace(' NA',-99,inplace=True)
df['age'] = df['age'].astype(np.int8)

df['antiguedad'].replace('     NA',-99,inplace=True)
df['antiguedad'] = df['antiguedad'].astype(np.int8)

df['renta'].replace('         NA',-99,inplace=True)
df['renta'].fillna(-99, inplace=True)
df['renta'] = df['renta'].astype(float).astype(np.int8)

df['indrel_1mes'].replace('P',5,inplace=True)
df['indrel_1mes'].fillna(-99, inplace=True)
df['indrel_1mes'] = df['indrel_1mes'].astype(float).astype(np.int8)

features += ['age','antiguedad','renta','ind_nuevo','indrel','indrel_1mes','ind_actividad_cliente']

### 2. 피처 엔지니어링
#### 24개의 제품 변수에 대해서 1개월전 보유 여부에 대한 파생 변수 생성

In [ ]:
# 월, 년 정보 추출
df['fecha_dato_dt'] = pd.to_datetime(df['fecha_dato'])
df['fecha_alta_month'] = df['fecha_dato_dt'].dt.month
df['fecha_alta_year'] = df['fecha_dato_dt'].dt.year

df.drop(['fecha_dato_dt'], axis=1 , inplace =True)
features += ['fecha_alta_month','fecha_alta_year']

In [ ]:
df['ult_fec_cli_1t_dt'] = pd.to_datetime(df['ult_fec_cli_1t'])
df['ult_fec_cli_1t_month'] = df['ult_fec_cli_1t_dt'].dt.month
df['ult_fec_cli_1t_year'] = df['ult_fec_cli_1t_dt'].dt.year

df.drop(['ult_fec_cli_1t_dt'], axis=1 , inplace =True)
features += ['ult_fec_cli_1t_month','ult_fec_cli_1t_year']

In [ ]:
# 나머지 결측치 -99로 처리
df.fillna(-99, inplace = True)

In [ ]:
# 날짜를 숫자로 변환(15년 2월은 2, 16년 2월은 12+2=14)
def date_to_int(str_date):
    Y, M, D = [int(a) for a in str_date.strip().split('-')]
    int_date = (Y-2015)*12 + M
    return int_date

df['int_date'] = df['fecha_dato'].map(date_to_int).astype(np.int8)

In [ ]:
# int_date에 1을 더한 df를 복사하여 합치기 - 기존과 지난달의 데이터 비교 가능
df_lag = df.copy()
df_lag.columns = [col+'_prev' if col not in ['ncodpers','int_date'] else col for col in df.columns]
df_lag['int_date'] += 1

In [ ]:
df_trn = df.merge(df_lag, on=['ncodpers','int_date'], how='left')

In [ ]:
# 필요없는 메모리 삭제
del df, df_lag

In [ ]:
# 지난 달의 제품정보가 없다면 0으로 대체
for prod in prods:
    prev = prod + '_prev'
    df_trn[prev].fillna(0, inplace=True)
df_trn.fillna(-99, inplace=True)

In [ ]:
# lag-1 변수 추가
features += [feature + '_prev' for feature in features] # 고객변수의 1개월 전 데이터 (prev)
features += [prod + '_prev' for prod in prods] # 제품변수의 1개월 전 데이터 (prev)

In [ ]:
pd.set_option('display.max_columns',None)
df_trn.head()

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,ult_fec_cli_1t,indrel_1mes,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,tipodom,cod_prov,nomprov,ind_actividad_cliente,renta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1,fecha_alta_month,fecha_alta_year,ult_fec_cli_1t_month,ult_fec_cli_1t_year,int_date,fecha_dato_prev,ind_empleado_prev,pais_residencia_prev,sexo_prev,age_prev,fecha_alta_prev,ind_nuevo_prev,antiguedad_prev,indrel_prev,ult_fec_cli_1t_prev,indrel_1mes_prev,tiprel_1mes_prev,indresi_prev,indext_prev,conyuemp_prev,canal_entrada_prev,indfall_prev,tipodom_prev,cod_prov_prev,nomprov_prev,ind_actividad_cliente_prev,renta_prev,segmento_prev,ind_ahor_fin_ult1_prev,ind_aval_fin_ult1_prev,ind_cco_fin_ult1_prev,ind_cder_fin_ult1_prev,ind_cno_fin_ult1_prev,ind_ctju_fin_ult1_prev,ind_ctma_fin_ult1_prev,ind_ctop_fin_ult1_prev,ind_ctpp_fin_ult1_prev,ind_deco_fin_ult1_prev,ind_deme_fin_ult1_prev,ind_dela_fin_ult1_prev,ind_ecue_fin_ult1_prev,ind_fond_fin_ult1_prev,ind_hip_fin_ult1_prev,ind_plan_fin_ult1_prev,ind_pres_fin_ult1_prev,ind_reca_fin_ult1_prev,ind_tjcr_fin_ult1_prev,ind_valo_fin_ult1_prev,ind_viv_fin_ult1_prev,ind_nomina_ult1_prev,ind_nom_pens_ult1_prev,ind_recibo_ult1_prev,fecha_alta_month_prev,fecha_alta_year_prev,ult_fec_cli_1t_month_prev,ult_fec_cli_1t_year_prev
0,2016-01-28,1432296,0,0,0,20,2015-08-07,1.0,5,1.0,-99,1,0,0,0,-99,0,0,0,3.0,0,0.0,-71,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2016,-99.0,-99.0,13,-99,-99.0,-99.0,-99.0,-99.0,-99,-99.0,-99.0,-99.0,-99,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-99.0,-99.0,-99.0,-99.0
1,2016-01-28,1432294,0,0,1,25,2015-08-07,1.0,5,1.0,-99,1,0,0,1,-99,0,0,0,12.0,1,1.0,-127,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2016,-99.0,-99.0,13,-99,-99.0,-99.0,-99.0,-99.0,-99,-99.0,-99.0,-99.0,-99,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-99.0,-99.0,-99.0,-99.0
2,2016-01-28,1432292,0,0,0,23,2015-08-07,1.0,5,1.0,-99,1,0,0,0,-99,0,0,0,3.0,0,0.0,-18,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2016,-99.0,-99.0,13,-99,-99.0,-99.0,-99.0,-99.0,-99,-99.0,-99.0,-99.0,-99,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-99.0,-99.0,-99.0,-99.0
3,2016-01-28,1432297,0,0,1,20,2015-08-07,1.0,5,1.0,-99,1,0,0,1,-99,0,0,0,45.0,2,0.0,-118,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2016,-99.0,-99.0,13,-99,-99.0,-99.0,-99.0,-99.0,-99,-99.0,-99.0,-99.0,-99,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-99.0,-99.0,-99.0,-99.0
4,2016-01-28,1432280,0,0,0,20,2015-08-07,1.0,5,1.0,-99,1,0,0,1,-99,0,0,0,45.0,2,0.0,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2016,-99.0,-99.0,13,-99,-99.0,-99.0,-99.0,-99.0,-99,-99.0,-99.0,-99.0,-99,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-99.0,-99.0,-99.0,-99.0


### 3. 머신러닝 모델 학습
xgboost 사용
베이스라인의 모델 간소화를 위해 2016년만 사용

In [ ]:
# 훈련 및 검증 데이터, 테스트 데이터 분리
use_dates = ['2016-01-28','2016-02-28','2016-03-28''2016-04-28','2016-05-28'] 
trn = df_trn[df_trn['fecha_dato'].isin(use_dates)]
tst = df_trn[df_trn['fecha_dato'] == '2016-06-28']

del df_trn

In [ ]:
# 훈련 및 검증 데이터에서 신규 구매 건수만 추출
X = []
Y = []
for i, prod in enumerate(prods):
    prev = prod + '_prev'
    prX = trn[(trn[prod] == 1) & (trn[prev] == 0)]
    prY = np.zeros(prX.shape[0], dtype=np.int8) + i # ex. 다섯번째 신규구매가 100건이라면 4가 100번 반복되는 배열이 만들어짐
    X.append(prX) # n번째 제품변수에 대한 신규 구매 건수
    Y.append(prY) # n번째임을 숫자로 표현

XY = pd.concat(X) # 24개의 모든 제품 변수에 대한 신규 구매 건수 추출 
Y = np.hstack(Y) # 다 옆으로 이어붙임
XY['y'] = Y # 각 신규 구매 건수에 어떤 제품 변수인지 표시

In [ ]:
# 신규 구매 건수에 대한 훈련, 검증데이터 분리
vld_date = '2016-05-28'
XY_trn = XY[XY['fecha_dato'] != vld_date]
XY_vld = XY[XY['fecha_dato'] == vld_date]

In [ ]:
param = {
    'booster' : 'gbtree',
    'max_depth' : 8,
    'nthread' : 4,
    'num_class' : len(prods),
    'objective' : 'multi:softprob',
    'silent' : 1,
    'eval_metric' : 'mlogloss',
    'eta' : 0.1,
    'min_child_weight' : 10,
    'colsample_bytree' : 0.8,
    'colsample_bylevel' : 0.9,
    'seed' : 2018
}

# 모델학습을 위한 데이터 형태 변환
X_trn = XY_trn[features].values # 데이터프레임을 행렬로 변환
Y_trn = XY_trn['y'].values
dtrn = xgb.DMatrix(X_trn, label=Y_trn, feature_names = features)

X_vld = XY_vld[features].values # 데이터프레임을 행렬로 변환
Y_vld = XY_vld['y'].values
dvld = xgb.DMatrix(X_vld, label=Y_vld, feature_names = features)

# 모델 학습
model = xgb.train(param, dtrn, num_boost_round=1000, evals =[(dtrn,'train'),(dvld,'eval')], early_stopping_rounds=20) 
best_ntree_limit = model.best_ntree_limit

# 시간이 너무 올려걸림...ㅠ

[16:35:32] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-37/xgboost/src/learner.cc:627: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	train-mlogloss:2.78543	eval-mlogloss:2.72483
[1]	train-mlogloss:2.60397	eval-mlogloss:2.52092
[2]	train-mlogloss:2.47224	eval-mlogloss:2.35728
[3]	train-mlogloss:2.37123	eval-mlogloss:2.23017
[4]	train-mlogloss:2.28431	eval-mlogloss:2.12078
[5]	train-mlogloss:2.21186	eval-mlogloss:2.02380
[6]	train-mlogloss:2.15050	eval-mlogloss:1.94875
[7]	train-mlogloss:2.09580	eval-mlogloss:1.88155
[8]	train-mlogloss:2.04790	eval-mlogloss:1.81721
[9]	train-mlogloss:2.00645	eval-mlogloss:1.76461
[10]	train-mlogloss:1.96958	eval-mlogloss:1.71295
[11]	tr

KeyboardInterrupt: 

In [ ]:
X_all = XY[features].values
Y_all = XY['y'].values

# 전체 데이터로 행렬 만들기
dall = xgb.DMatrix(X_all, label=Y_all, feature_names = features)

# 늘어난 데이터 양만큼 트리 개수 늘리기
best_ntree_limit = int(best_ntree_limit*(len(XY_trn)+len(XY_vld))/len(XY_trn))

# 모델 재학습
model = xgb.train(param, dall, num_boost_round = best_ntree_limit, evals=[(dall,'train')])

In [ ]:
X_tst = tst[features].values
dtst = xgb.Dmatrix(X_tst, feature_names=features)
preds_tst = model.predict(dtst, ntree_limit = best_ntree_limit)

ncodpers_tst = tst['ncodpers'].values
preds = [ 1 if x > 0.5 else 0 for x in preds_tst ]

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score

def get_clf_eval(Y_vld, pred=None, preds_tst=None):
    confusion = confusion_matrix( Y_vld, pred)
    accuracy = accuracy_score(Y_vld , pred)
    precision = precision_score(Y_vld , pred)
    recall = recall_score(Y_vld , pred)
    f1 = f1_score(Y_vld,pred)
    # ROC-AUC 추가 
    roc_auc = roc_auc_score(Y_vld, preds_tst)
    print('오차 행렬')
    print(confusion)
    # ROC-AUC print 추가
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}, F1: {3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))

get_clf_eval(Y_vld , preds, preds_tst)

NameError: name 'preds' is not defined